 # Image Colorization using Deep Transfer Learning

### The goal of this project is to construct the most effective convolutional neural network for converting grayscale images into RGB images. In image colorization, each pixel of a target grayscale image is given a colour. For image colorization, we will apply an entirely automated data-driven method known as autoencoders.  We'll utilise the VGG16 model as a feature extractor.

#### Importing libraries


In [6]:
import keras
import tensorflow as tf

In [8]:
"Image colorization using Autoencoders. Transfer learning using VGG."
"Importing Necessary Libraries"

from keras.layers import Conv2D, UpSampling2D, Input
from keras.models import Sequential, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, gray2rgb
from skimage.transform import resize
from skimage.io import imsave
import numpy as np

import keras
import os

#### Loading the pretrained VGG16 model

In [9]:
"Replacing the encoder part with Feature Extraxtor of VGG"
from keras.applications.vgg16 import VGG16
vggmodel = VGG16()
newmodel = Sequential() 
#num = 0
for i, layer in enumerate(vggmodel.layers):
    if i<19:          #Only up to 19th layer to include feature extraction only
      newmodel.add(layer)
newmodel.summary()
for layer in newmodel.layers:
  layer.trainable=False  #We don't want to train these layers again, so False. 

553467096/553467096 [==============================] - 230s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                     

#### Loading and preprocessing dataset

In [10]:
"""
VGG16 is expecting an image of 3 dimension with size 224x224 as an input, 
in preprocessing we have to scale all images to 224 instead of 256
"""
ROOT_DIR = 'path_to_dataset' # give directory path to colored images dataset
#Normalize images - divide by 255
train_datagen = ImageDataGenerator(rescale=1. / 255)

train = train_datagen.flow_from_directory(ROOT_DIR, target_size=(224, 224), batch_size=1000, class_mode=None)

Found 7134 images belonging to 2 classes.


###### Lab is a color space commonly used in digital image processing and computer vision. It represents colors based on three components: L for lightness, a for the green-red color component, and b for the blue-yellow color component. The LAB color space is designed to approximate human perception of color more accurately than other color spaces like RGB.

In [11]:
"Convert from RGB to Lab: LAB image is a grey image in L channel and all color info stored in A and B channels"
X =[]
Y =[]
for img in train[0]:
  try:
      lab = rgb2lab(img)
      X.append(lab[:,:,0]) 
      Y.append(lab[:,:,1:] / 128) #A and B values range from -127 to 128, 
      #so we divide the values by 128 to restrict values to between -1 and 1.
  except:
     print('error')
X = np.array(X)
Y = np.array(Y)
X = X.reshape(X.shape+(1,)) #dimensions to be the same for X and Y
print(X.shape)
print(Y.shape)

(1000, 224, 224, 1)
(1000, 224, 224, 2)


In [12]:
#now we have one channel of L in each layer but, VGG16 is expecting 3 dimension, 
#so we repeated the L channel two times to get 3 dimensions of the same L channel
vggfeatures = []
for i, sample in enumerate(X):
  sample = gray2rgb(sample)
  sample = sample.reshape((1,224,224,3))
  prediction = newmodel.predict(sample)
  prediction = prediction.reshape((7,7,512))
  vggfeatures.append(prediction)
vggfeatures = np.array(vggfeatures)
print(vggfeatures.shape)

1/1 [==============================] - 0s 104ms/step


1/1 [==============================] - 0s 119ms/step


1/1 [==============================] - 0s 114ms/step


1/1 [==============================] - 0s 110ms/step


1/1 [==============================] - 0s 113ms/step


1/1 [==============================] - 0s 112ms/step


1/1 [==============================] - 0s 116ms/step
(1000, 7, 7, 512)


#### Adding custom decoder layers on top of VGG16 model

In [13]:
#Decoder
model = Sequential()

model.add(Conv2D(256, (3,3), activation='relu', padding='same', input_shape=(7,7,512)))
model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(16, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))
model.add(UpSampling2D((2, 2)))
model.summary()



Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 7, 7, 256)         1179904   
                                                                 
 conv2d_1 (Conv2D)           (None, 7, 7, 128)         295040    
                                                                 
 up_sampling2d (UpSampling2D  (None, 14, 14, 128)      0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 14, 14, 64)        73792     
                                                                 
 up_sampling2d_1 (UpSampling  (None, 28, 28, 64)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 28, 28, 32)       

#### Compiling and training the model

In [14]:
model.compile(optimizer='Adam', loss='mse' , metrics=['accuracy'])
model.fit(vggfeatures, Y, verbose=1, epochs=2000, batch_size=16)


Epoch 1/2000
63/63 [==============================] - 11s 166ms/step - loss: 0.0743 - accuracy: 0.5511
Epoch 2/2000
63/63 [==============================] - 11s 172ms/step - loss: 0.0093 - accuracy: 0.6796
Epoch 3/2000
63/63 [==============================] - 11s 175ms/step - loss: 0.0084 - accuracy: 0.6966
Epoch 4/2000
63/63 [==============================] - 11s 174ms/step - loss: 0.0079 - accuracy: 0.7073
Epoch 5/2000
63/63 [==============================] - 11s 172ms/step - loss: 0.0072 - accuracy: 0.7182
Epoch 6/2000
63/63 [==============================] - 11s 172ms/step - loss: 0.0069 - accuracy: 0.7271
Epoch 7/2000
63/63 [==============================] - 11s 172ms/step - loss: 0.0064 - accuracy: 0.7383
Epoch 8/2000
63/63 [==============================] - 11s 172ms/step - loss: 0.0060 - accuracy: 0.7461
Epoch 9/2000
63/63 [==============================] - 11s 181ms/step - loss: 0.0057 - accuracy: 0.7552
Epoch 10/2000
63/63 [==============================] - 11s 180ms/step - l

63/63 [==============================] - 12s 190ms/step - loss: 0.0025 - accuracy: 0.8249
Epoch 80/2000
63/63 [==============================] - 12s 190ms/step - loss: 0.0025 - accuracy: 0.8243
Epoch 81/2000
63/63 [==============================] - 12s 186ms/step - loss: 0.0026 - accuracy: 0.8247
Epoch 82/2000
63/63 [==============================] - 11s 181ms/step - loss: 0.0027 - accuracy: 0.8203
Epoch 83/2000
63/63 [==============================] - 11s 181ms/step - loss: 0.0030 - accuracy: 0.8104
Epoch 84/2000
63/63 [==============================] - 11s 182ms/step - loss: 0.0030 - accuracy: 0.8111
Epoch 85/2000
63/63 [==============================] - 11s 180ms/step - loss: 0.0033 - accuracy: 0.8004
Epoch 86/2000
63/63 [==============================] - 11s 180ms/step - loss: 0.0029 - accuracy: 0.8147
Epoch 87/2000
63/63 [==============================] - 11s 180ms/step - loss: 0.0027 - accuracy: 0.8216
Epoch 88/2000
63/63 [==============================] - 11s 180ms/step - loss: 

63/63 [==============================] - 11s 181ms/step - loss: 0.0022 - accuracy: 0.8418
Epoch 158/2000
63/63 [==============================] - 11s 181ms/step - loss: 0.0023 - accuracy: 0.8387
Epoch 159/2000
63/63 [==============================] - 11s 182ms/step - loss: 0.0022 - accuracy: 0.8433
Epoch 160/2000
63/63 [==============================] - 11s 180ms/step - loss: 0.0021 - accuracy: 0.8465
Epoch 161/2000
63/63 [==============================] - 12s 183ms/step - loss: 0.0020 - accuracy: 0.8495
Epoch 162/2000
63/63 [==============================] - 11s 181ms/step - loss: 0.0020 - accuracy: 0.8518
Epoch 163/2000
63/63 [==============================] - 11s 181ms/step - loss: 0.0020 - accuracy: 0.8481
Epoch 164/2000
63/63 [==============================] - 11s 181ms/step - loss: 0.0020 - accuracy: 0.8517
Epoch 165/2000
63/63 [==============================] - 11s 182ms/step - loss: 0.0019 - accuracy: 0.8532
Epoch 166/2000
63/63 [==============================] - 11s 181ms/step

63/63 [==============================] - 12s 183ms/step - loss: 0.0022 - accuracy: 0.8407
Epoch 236/2000
63/63 [==============================] - 11s 182ms/step - loss: 0.0020 - accuracy: 0.8514
Epoch 237/2000
63/63 [==============================] - 11s 181ms/step - loss: 0.0020 - accuracy: 0.8463
Epoch 238/2000
63/63 [==============================] - 11s 181ms/step - loss: 0.0021 - accuracy: 0.8507
Epoch 239/2000
63/63 [==============================] - 11s 182ms/step - loss: 0.0019 - accuracy: 0.8529
Epoch 240/2000
63/63 [==============================] - 11s 181ms/step - loss: 0.0018 - accuracy: 0.8571
Epoch 241/2000
63/63 [==============================] - 11s 182ms/step - loss: 0.0018 - accuracy: 0.8587
Epoch 242/2000
63/63 [==============================] - 12s 184ms/step - loss: 0.0017 - accuracy: 0.8610
Epoch 243/2000
63/63 [==============================] - 12s 188ms/step - loss: 0.0017 - accuracy: 0.8628
Epoch 244/2000
63/63 [==============================] - 11s 182ms/step

63/63 [==============================] - 11s 182ms/step - loss: 0.0018 - accuracy: 0.8586
Epoch 314/2000
63/63 [==============================] - 12s 183ms/step - loss: 0.0018 - accuracy: 0.8586
Epoch 315/2000
63/63 [==============================] - 11s 182ms/step - loss: 0.0018 - accuracy: 0.8595
Epoch 316/2000
63/63 [==============================] - 11s 182ms/step - loss: 0.0017 - accuracy: 0.8623
Epoch 317/2000
63/63 [==============================] - 12s 183ms/step - loss: 0.0017 - accuracy: 0.8620
Epoch 318/2000
63/63 [==============================] - 12s 183ms/step - loss: 0.0017 - accuracy: 0.8630
Epoch 319/2000
63/63 [==============================] - 12s 183ms/step - loss: 0.0017 - accuracy: 0.8590
Epoch 320/2000
63/63 [==============================] - 11s 182ms/step - loss: 0.0016 - accuracy: 0.8643
Epoch 321/2000
63/63 [==============================] - 12s 183ms/step - loss: 0.0016 - accuracy: 0.8673
Epoch 322/2000
63/63 [==============================] - 11s 182ms/step

...

63/63 [==============================] - 11s 178ms/step - loss: 0.0015 - accuracy: 0.8700
Epoch 1632/2000
63/63 [==============================] - 11s 178ms/step - loss: 0.0015 - accuracy: 0.8676
Epoch 1633/2000
63/63 [==============================] - 11s 178ms/step - loss: 0.0014 - accuracy: 0.8732
Epoch 1634/2000
63/63 [==============================] - 11s 178ms/step - loss: 0.0014 - accuracy: 0.8733
Epoch 1635/2000
63/63 [==============================] - 11s 178ms/step - loss: 0.0015 - accuracy: 0.8714
Epoch 1636/2000
63/63 [==============================] - 11s 177ms/step - loss: 0.0016 - accuracy: 0.8669
Epoch 1637/2000
63/63 [==============================] - 11s 178ms/step - loss: 0.0016 - accuracy: 0.8680
Epoch 1638/2000
63/63 [==============================] - 11s 178ms/step - loss: 0.0017 - accuracy: 0.8653
Epoch 1639/2000
63/63 [==============================] - 11s 180ms/step - loss: 0.0015 - accuracy: 0.8665
Epoch 1640/2000
63/63 [==============================] - 11s 1

63/63 [==============================] - 11s 177ms/step - loss: 0.0014 - accuracy: 0.8715
Epoch 1709/2000
63/63 [==============================] - 11s 177ms/step - loss: 0.0015 - accuracy: 0.8703
Epoch 1710/2000
63/63 [==============================] - 11s 177ms/step - loss: 0.0015 - accuracy: 0.8726
Epoch 1711/2000
63/63 [==============================] - 11s 177ms/step - loss: 0.0014 - accuracy: 0.8721
Epoch 1712/2000
63/63 [==============================] - 11s 177ms/step - loss: 0.0015 - accuracy: 0.8707
Epoch 1713/2000
63/63 [==============================] - 11s 177ms/step - loss: 0.0015 - accuracy: 0.8683
Epoch 1714/2000
63/63 [==============================] - 11s 178ms/step - loss: 0.0015 - accuracy: 0.8719
Epoch 1715/2000
63/63 [==============================] - 11s 177ms/step - loss: 0.0014 - accuracy: 0.8751
Epoch 1716/2000
63/63 [==============================] - 11s 177ms/step - loss: 0.0014 - accuracy: 0.8757
Epoch 1717/2000
63/63 [==============================] - 11s 1

63/63 [==============================] - 11s 182ms/step - loss: 0.0014 - accuracy: 0.8738
Epoch 1786/2000
63/63 [==============================] - 11s 178ms/step - loss: 0.0014 - accuracy: 0.8746
Epoch 1787/2000
63/63 [==============================] - 11s 178ms/step - loss: 0.0014 - accuracy: 0.8771
Epoch 1788/2000
63/63 [==============================] - 11s 179ms/step - loss: 0.0014 - accuracy: 0.8762
Epoch 1789/2000
63/63 [==============================] - 11s 177ms/step - loss: 0.0013 - accuracy: 0.8777
Epoch 1790/2000
63/63 [==============================] - 11s 180ms/step - loss: 0.0013 - accuracy: 0.8782
Epoch 1791/2000
63/63 [==============================] - 11s 178ms/step - loss: 0.0013 - accuracy: 0.8785
Epoch 1792/2000
63/63 [==============================] - 11s 179ms/step - loss: 0.0014 - accuracy: 0.8763
Epoch 1793/2000
63/63 [==============================] - 11s 178ms/step - loss: 0.0014 - accuracy: 0.8743
Epoch 1794/2000
63/63 [==============================] - 11s 1

63/63 [==============================] - 12s 194ms/step - loss: 0.0015 - accuracy: 0.8713
Epoch 1863/2000
63/63 [==============================] - 12s 193ms/step - loss: 0.0015 - accuracy: 0.8703
Epoch 1864/2000
63/63 [==============================] - 11s 179ms/step - loss: 0.0019 - accuracy: 0.8578
Epoch 1865/2000
63/63 [==============================] - 12s 186ms/step - loss: 0.0018 - accuracy: 0.8587
Epoch 1866/2000
63/63 [==============================] - 12s 191ms/step - loss: 0.0016 - accuracy: 0.8666
Epoch 1867/2000
63/63 [==============================] - 12s 192ms/step - loss: 0.0016 - accuracy: 0.8674
Epoch 1868/2000
63/63 [==============================] - 12s 194ms/step - loss: 0.0015 - accuracy: 0.8704
Epoch 1869/2000
63/63 [==============================] - 13s 199ms/step - loss: 0.0015 - accuracy: 0.8710
Epoch 1870/2000
63/63 [==============================] - 12s 190ms/step - loss: 0.0015 - accuracy: 0.8704
Epoch 1871/2000
63/63 [==============================] - 12s 1

63/63 [==============================] - 11s 178ms/step - loss: 0.0013 - accuracy: 0.8786
Epoch 1940/2000
63/63 [==============================] - 11s 177ms/step - loss: 0.0014 - accuracy: 0.8777
Epoch 1941/2000
63/63 [==============================] - 11s 177ms/step - loss: 0.0014 - accuracy: 0.8762
Epoch 1942/2000
63/63 [==============================] - 11s 178ms/step - loss: 0.0015 - accuracy: 0.8726
Epoch 1943/2000
63/63 [==============================] - 11s 178ms/step - loss: 0.0015 - accuracy: 0.8691
Epoch 1944/2000
63/63 [==============================] - 11s 177ms/step - loss: 0.0015 - accuracy: 0.8732
Epoch 1945/2000
63/63 [==============================] - 11s 177ms/step - loss: 0.0014 - accuracy: 0.8753
Epoch 1946/2000
63/63 [==============================] - 11s 176ms/step - loss: 0.0015 - accuracy: 0.8740
Epoch 1947/2000
63/63 [==============================] - 11s 177ms/step - loss: 0.0017 - accuracy: 0.8607
Epoch 1948/2000
63/63 [==============================] - 11s 1

In [22]:
model.save(ROOT_DIR_1+'colorize_autoencoder_2000.model')

INFO:tensorflow:Assets written to: C:/Users/Urvi Latnekar/Desktop/colorize_autoencoder_2000.model\assets


INFO:tensorflow:Assets written to: C:/Users/Urvi Latnekar/Desktop/colorize_autoencoder_2000.model\assets


In [20]:
ROOT_DIR_1 ='path_to_saved_model'

In [23]:
#Predicting using saved model.
model = tf.keras.models.load_model(ROOT_DIR_1+'colorize_autoencoder_2000.model',
                                   custom_objects=None,
                                   compile=True)


In [24]:
testpath = 'path_to_test_dataset'
files = os.listdir(testpath)
files

['t1.jpg', 't2.jpg', 't3.jpg', 't4.jpg', 't5.jpg']

In [26]:
for idx, file in enumerate(files):
    test = img_to_array(load_img(testpath+file))
    test = resize(test, (224,224), anti_aliasing=True)
    test*= 1.0/255
    lab = rgb2lab(test)
    l = lab[:,:,0]
    L = gray2rgb(l)
    L = L.reshape((1,224,224,3))
    #print(L.shape)
    vggpred = newmodel.predict(L)
    ab = model.predict(vggpred)
    #print(ab.shape)
    ab = ab*128
    cur = np.zeros((224, 224, 3))
    cur[:,:,0] = l
    cur[:,:,1:] = ab
    imsave('path_to_saved_output'+str(idx)+".jpg", lab2rgb(cur))

1/1 [==============================] - 0s 37ms/step


1/1 [==============================] - 0s 41ms/step


1/1 [==============================] - 0s 30ms/step


1/1 [==============================] - 0s 33ms/step


1/1 [==============================] - 0s 31ms/step


In [ ]:
!pip freeze > ImageColoring/MyDrive/ImageColoring/requirements.txt